# SMPC Protocols Explanation 
This notebook will give you an overview and a very quick explanation over the different SMPC protocols that are currently implemented in PySyft. It'll also elaborate on **what kind of Machine Learning you can conduct in an encrypted fashion using them**, along a comparison of their resulting performance to each other and to the non-encrypted scenario. The protocol explanations should mainly serve to give you a high-level understanding of what the main **crypto-slang** stands for, how the terms relate to each other and what ressources serve as good starting points to dig deeper! 


## Quick recap - What is SMPC encryption?
As a quick recap "SMPC" stands for **Secure Multi-Party Computation** and constitutes a form of encryption which can be used for Machine Learning (i.e. it is possible to do calculations on encrypted data) leveraging a network of min. 2 different servers. These are being trusted of not collaborating with each other and as a whole work as a **trusted execution environment** on which sensitive calculations such as model inference, training, etc. can be done without the model or the data being disclosed to any party besides the respective owner. 

## General concepts 
* Important concepts that all protocols are based upon - *in brief:* 
  * **A Public value,** is considered data (e.g. input from the data-sources) which is known by all parties. 
  * **A Private value,** is considered data which is secured through additive secret sharing, only the owner knows the true value. 
  * **Masking:** To share a private value publicily (e.g. as necessary part in a protcol) it has to be masked first. This typically simply involes adding a random number to the value and projecting it onto a fixed set of numbers, a so called ring. <br>  *masked_value = (private_value - random_numb) % upper absolute value of set of numbers.* For more info on that checkout the Udacity Beginner Tutorials by Andrew Trask. 

In [ ]:
# Setup of example use-case problem 
# Serves to compare best models that can still be encrypted using the different protocols +
# Comparison to non-encrypted computation. (Only inference, we use pre-trained models)

# 1. SPDZ Protocol 
* **Encryption on n parties possible (n >= 2)**
* Basis for advanced protocols secureNN and FSS. 
* *In depth material:*
  * [Bristol Cryptography Blog Series](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html)
  * [Morten Dahl's Blog](https://mortendahl.github.io/2017/09/03/the-spdz-protocol-part1/)
  * [PySyft Code](https://github.com/OpenMined/PySyft/blob/master/syft/frameworks/torch/mpc/spdz.py)

### General concept 
There are two types of SMPC: secret-sharing-based SMPC and circuit-gabling-based SMPC. 
The two protocols (secureNN and FSS) that are implemented in PySyft to date are both based on the **SPDZ protocol**, 
which is based on *additive secret-sharing* (the first category) For more information on what *additive secret-sharing* is, check out the tutorials by Andrew Trask as part of the Udacity Private and Secure AI Course.<br>
SPDZ is a very widely used protocol for computing linear functions in an encrypted fashion, meaning it can be used to compute sums and multiplication of encrypted variables. <span style="color:red;">The sum of encrypted variables simply consists of each server summing locally its shares of the private variables, which then leads to each server having a share of the sum of the private variables, thus together secret-sharing the sum of the private variables.</span> The multiplication of two encrypted variables however is done using so called **"beaver multiplication triples"** - three randomely generated numbers - which make the encrypted multiplication very efficient compared to other appraoches (e.g. garbled circuits)
<br>
*Why more efficient?* See as a brief introduction into this topic the 'Extra: Beaver Multiplication Triples' section or to dig deeper see the ressources mentioned just above. 
<br>
<br>
The high-level procedure of SPDZ-based protocols is as follow: (based on definition from the [Bristol Cryptography Blog](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html))
   1. Parties secret-share their inputs at the beginning (one crypto-provider exists to generate beaver triples, etc.)
   2. Parties compute mul. and sum. locally (using only their share of the private variables). <span style="color:red;">*By design they don't have to communicate to each other while doing the sums and products locally. Only at the end they share their end-result. This reduces communication complexcity significantly.*</span> 
   3. At the end parties reveal the result of their calcultaion by sending their share to all others and adding all parts up (each server does that and thus knows the final result)

### Extra: Beaver Multiplication Triples
Beaver Triples are simply put randomely generated numbers which are also shared among all different workers and are used to mask (as described above) the input variables to the multiplication, so that they can be publicly shared. Then they can be only used to compute the product (the end-result) by simply computing a simple equation out of all masked input variables, which cancels out the random masks and reveals the product. So effectively no further communcations between the workers is necessary during the computation itself. Now concerning the overall efficiency of the protocol you might think that although the communication-complexity is low we still need to generate new Beaver Triples for each multiplication? (because of security issues they can only be used for one multiplication) <br>
For this reason however, the SPDZ protocol is parted into an 'offline' and an 'online' phase. The 'offline' phase consists of randomely generating all necessary "crypto-primitives" - e.g. the Beaver Triples - and can essentially be executed independently of specific inputs of the multiplication, i.e. before the actual multiplications. Thus, given we have generated enough Beaver Triples beforehand, during the 'online' phase we can compute as much multiplications as we have pre-generated Beaver Triples. *This makes the SPDZ protocol very efficient for linear computations!*

### Extra: Crypto-Store
  * As mentioned above a key feature of the SPDZ-protocol is its splitting of the encrypted computation process into an *online* and an *offline* phase to allow for a significantly decreased execution time (given enough time for offline phase when no encrypted computations are being conducted)
  * As you probably know in PySyft there exist *worker* objects that have certain default attributes specified and set in `class BaseWorker(AbstractWorker)` ([code](https://github.com/OpenMined/PySyft/blob/c83e615a85bb8944245668d90582fb97c45e6e18/syft/workers/base.py#L48)). One of them is a so called `worker.crypto_store`. The *crypto_store* object is of type `class PrimitiveStorage`, which specifies a set of given functions which help the respective workers to manage crypto-primitives they need to participate in the respective crypto-protocol. (e.g. Beaver Triples for multiplication in the SPDZ-protocol)
  * Specifically there are two important methods of the crypto_store object. For the *crypto-provider* - the server which serves as a trusted, neutral participant of the protocol - the `crypto_provider.crypto_store.provide_primitives(...)` ([code](https://github.com/OpenMined/PySyft/blob/c83e615a85bb8944245668d90582fb97c45e6e18/syft/frameworks/torch/mpc/primitives.py#L161)) method generates and sends crypto-primitives such as Beaver Triples to participating workers. <br> For "normal" participating workers the `worker.crypto_store.get_keys(...)` ([code](https://github.com/OpenMined/PySyft/blob/c83e615a85bb8944245668d90582fb97c45e6e18/syft/frameworks/torch/mpc/primitives.py#L52)) method takes care of receiving and storing the crypto primitives for later usage during the protocol.

### Extra: Garbled Circuits 
Maybe you've read this term a couple of times on the OpenMined workspace or in SMPC-encryption discussions - *on a high-level what is circuit-gabling-based SMPC, or more specifically what are'Garbled Circuits?* <br>
Garbled Circuits is a protocol which works as **2-Party-Computation** (i.e. only two servers are involved) The name comes from the method which is used to encrypt a function: a function is represented as a **circuit** consisting of different logical gates (e.g. AND, XOR, etc.) This circuit can be described with a truthtable which indicates at what inputs, what outputs follow. As part of the encryption process the rows of the truthtable are re-ordered arbitrarily which leads to the name "garbled" circuits. <br> 
The [details about the protocol](https://en.wikipedia.org/wiki/Garbled_circuit) are rather straight forward, the important thing to note is that this technique is **very flexible** as essentially all functions can be encrypted using this method, but also **very inefficient**. That's why often the Garbled Circuits protocol is mainly used on-top of more optimized protocols (such as e.g. SPDZ) to extend the variety of functions compatible with these protocols while not slowing down the computation too much for most computations. <br>
It has to be noted that nevertheless, researchers try to find alternative, more efficient protocols which work withouut Garbled Circuits at the cost of supporting only a lower variety of functions. *Examples for protocols especially for computations that are popular in Machine Learning Models are secureNN and FSS which PySyft supports and over which I'll go over in the following section!*

In [ ]:
# Implementation

# 2. SecureNN Protocol
* Introduced in [paper](https://www.researchgate.net/publication/334590018_SecureNN_3-Party_Secure_Computation_for_Neural_Network_Training): *SecureNN: 3-Party Secure Computation for Neural Network Training, by Sameer Wagh, Divya Gupta, and Nishanth Chandran, in Proceedings on Privacy Enhancing Technologies,  2019* 
* **Protocol is made for computation on 2 parties with 1 crypto-worker.** (There can still be multiple data-owners, but the computations need to be done on 2 servers - 2-party-additive-sharing)

## High-Level concept 
SecureNN uses the SPDZ protocol for linear layers (beaver triples, etc.) and contains multiple efficent protocols for common non-linearities, as further described below. Compared to ealier work SecureNN implements the computation of the non-linearities without the need of Garbled Circuits (as is the case for SecureML, which was considered the state-of-the-art ML protocol before SecureNN) This also means "interconversation protocols" to bridge between encoding needed for SPDZ and encoding needed for Garbled Circuits aren't necessary, decreasing computation time further. <br>
<span style="color:red;">*In general SecureNN is therefore a lot faster than SecureML and other garbled-circuit-based (for non-linearities) protocols.* </span>

## In detail 
### Possible computations - Low Level 
This isn't a general purpose protocol, which can compute all possible kinds of computations that are used in training NNs, but by giving up this flexibility (e.g. no usasge of garbled circuits) we gain efficiency which is a vital criteria for the real-world-applicability of an encryption protocol. 

* Matrix Multiplication (SPDZ)
* Select Share 
  * <span style="color:red;">Select one variable out of multiple private variables to be freshly masked and shared for a new computation. </span> Used for e.g. for maxpool (i.e. select max element out of kernel-elements)
* Private Compare 
  * <span style="color:red;">Compare public variable with private variable. Used to compute the ReLU function (ReLU = max(x,0))</span> Used for e.g. for computation of MSB (see below)
* Share Convert 
  * Convert private variables from one number space (a "ring") to another. <span style="color:red;">Remember during computation the numbers are plain integers encoded by a bit-sequence of length L. After "share convert" they are encoded as a bit-sequence of length L-1. </span> Used for e.g. for computation of derivative of ReLU. 
* Compute MSB (Most-Significant-Bit)
  * Efficient reading of the sign-bit (is input integer positive or negative) mainly to compute the derivative of the ReLU function. 

### Possible functions - High Level (Non exhaustive list)
Resulting from the possible low level computations the following standard models, optimizers and loss-functions can be used. *This should provide a useful summary, but doesn't claim to be an exhaustive list (feel free to add important items that can be composed out of the above mentioned possible low-level computations)

* **Model Architecture:** Includes the possibility for encrypted computation of the derivative (needed for backprop)
    * **Linear Layers** 
      * Matrix Multiplication and Convolutions (in CNNs e.g.)
      * Average Pooling 
      * Batch-Norm/Normalization (Division of two private variables in general is possible)
      * <span style="color:red;">Dropout</span> - *with help of select share, certain computation-results can be ignored. Or simply set some inputs to some neurons to zero.*
    * **Non-Linear Layers:** 
      * Max Pooling 
      * ReLU, Leaky ReLU, Piece-wise linear activation functions 
     
* **Optimizers:** 
  * SGD (with Momentum)
  * ADAM (Momentum + RMSProp) - *as devision is also possible (and elementwise-multiplication as well)*

* **Loss-functions:**
  * L1-Loss - *as max() is possible*
  * MSE - *linear computation and power can be computed with SPDZ* 
  * Hinge-Loss (Linear Classification with Soft-Margin-SVM) - *as max(0, t) is possible*
  * Cross-Entropy-Loss
  
<span style="color:red;">***Question:*** How is Cross-Entropy-Loss supported (mentioned in paper), Softmax, Log are not explicitly mentioned as compatible functions. Use of approximation?</span>

### Security Guarantees 
* Full Security includes **privacy and correctness**
* The following to gurantees hold for all settings where there is a majority of honest participating servers (**Not in dishonest majority setting!**)
1. **Full Security for semi-honest corruption of a server** 
  * Privacy and Correctness of the data is secured if a server is being *honest-but-curious.* Meaning that follows the protocol but tries to infer as much information about the data it sees as possible. 
2. **Privacy against malicious server** 
  * Even a server which doesn't follow the given protocol can't learn anything about the inputs and outputs of the other (honest) servers, *given that the majority of the participants are honest!* This is a common assumption for the malicious case because in a real setting deviating from the given protocol could be prevented with additional measures, as seen below. 
3. Potential Add-on: **Security with Abort**
  * Protection against malicious servers can be guaranteed by adding [MAC authentication](https://en.wikipedia.org/wiki/Message_authentication_code) to the protocol. This would allow to abort the protocol as soon as one of the servers doesn't follow the protocol anymore. 


### Performance Evaluation - Important Metrics 
* Important Metrics: 
  * Communication Complexity 
  * Round Complexity 
  
See picture from SecureNN paper. *"The function
Linearm,n,v denotes a matrix multiplication of dimen-
sion m × n with n × v. Conv2dm,i,f,o denotes a convo-
lutional layer with input m × m, i input channels, a
filter of size f × f, and o output channels. lD denotes
precision of bits. Maxpooln and DMPn denotes Maxpool
and its derivative over n elements. All commu- nication is measured for l−bit inputs and p 
denotes the field size (which is 67 in our case)"*

<span style="color:red;">***TODO:*** Do very small comparision with other existing protocols  + try to evaluate performance based on below use-case. More detailed comparison between secureNN and FSS (AriaNN implementation) and non-encrypted scenario at the end!</span><br><br>
<span style="color:red;">***TODO:*** Think whether Info-text and complexity table are helpful (Table 2)</span>

In [ ]:
# Example Implementation 

# 3. Functional Secret Sharing Protocol
* Base [paper](https://link.springer.com/content/pdf/10.1007/978-3-662-46803-6_12.pdf) (first introduction): *Function secret sharing. E. Boyle, N. Gilboa, and Y. Ishai. In EUROCRYPT 2015, pages 337–367, 2015.* 
* **Encryption with n parties possible (n >= 2)**

## High Level Concept 
As the SecureNN protocol the FSS protocol is also based on the SPDZ protocol for encrypted computation of *linear layers* and provides additional protocols for *non-linearities*. The fundamental difference in the FSS protocol is that instead of evaluating a public function (e.g. a ReLU activation function) at a private value (the secret shared data from the data-sources) it rather evaluates a private function at a public value. This is possible by first masking the private data (x - r mod Q) and then making it publicily available to obtain a public value and to then secret share the function. <br> 
*How exactly do you secret-share a function? Check out [Théo's Tutorial](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2011%20bis%20-%20Encrypted%20inference%20on%20ResNet-18.ipynb) for a nice intro on that!*
 

### Base Implementation 
The concept of "Function-Secret-Sharing" (FSS) was first introduced in the [paper](https://eprint.iacr.org/2018/707): *Function Secret Sharing - Improvements and Extensions, by Elette Boyle and Niv Gilboa and Yuval Ishai, 2018* (This is the updated version)

### AriaNN Implementation 
The PySyft Implementation is based on the [paper](https://arxiv.org/abs/2006.04593): *ARIANN: Low-Interaction Privacy-Preserving Deep Learning via Function Secret Sharing, by Théo Ryffel, David Pointcheval, Francis Bach, 2020* 


## In Detail

### Possible computations - Low Level 
Similar to SecureNN the current FSS protocol implemented in PySyft is focused on efficiency specifically for common computations in Machine Learning, thus leading to decreased flexibility but also allowing decreased computation time for relevant computations. <br>
The following computations are supported: 

* Matrix Multiplication (SPDZ)
* Equality Test 
  * It is checked whether a public value equals a private (i.e. shared) value 
* Comparison 
  * Inequality between a public value/expression and private value/expression


### Possible functions - High Level (Non exhaustive list)
Resulting from the possible low level computations the following standard models, optimizers and loss-functions can be used. *This should provide a useful summary, but doesn't claim to be an exhaustive list (feel free to add important items that can be composed out of the above mentioned possible low-level computations)

* **Model Architecture:** Includes the possibility for encrypted computation of the derivative (needed for backprop)
    * **Linear Layers** with SPDZ using beaver triples 
      * Matrix Multiplication and Convolutions (in CNNs e.g.)
      * Average Pooling 
      * Batch-Norm/Normalization (approx. with Newton Method) 
      * **NO** Dropout - <span style="color: red;">Why? Can't we just set the intermediate inputs to zero? (To turn off some neurons)</span>
    * **Non-Linear Layers:** mainly based on direct comparison capabilities
      * Max Pooling 
      * ReLU, Leaky ReLU, Piece-wise linear activation functions 
      * Argmax 
     
* **Optimizers:**
  * SGD (with Momentum)
  * **NO** ADAM

* **Loss-functions:**
  * L1-Loss 
  * MSE 
  * Hinge Loss
  * **NO** Cross-Entropy-Loss, Logistic Loss

### Security Guarantees 
* Full Security includes **privacy and correctness**
* The following to gurantees hold for all settings where there is a majority of honest participating servers (**Not in dishonest majority setting!**)
1. **Full Security for semi-honest corruption of a server** 
  * Privacy and Correctness of the data is secured if a server is being *honest-but-curious.* Meaning that follows the protocol but tries to infer as much information about the data it sees as possible. 
2. Potential Add-on: **Security with Abort**
  * Protection against malicious servers can be guaranteed by adding [MAC authentication](https://en.wikipedia.org/wiki/Message_authentication_code) to the protocol. This would allow to abort the protocol as soon as one of the servers doesn't follow the protocol anymore. 


### Performance Evaluation - Important Metrics 
* Important Metrics: 
  * Communication Complexity 
  * Round Complexity 
<span style="color:red;">***TODO:*** Think whether Info-text and complexity table are helpful</span>

In [ ]:
# Implementation

# 4. Other encryption protocols in PySyft 

## FALCON Protocol (Future)
* Paper + Code 

## Potential Changes (Temporary)
* Add explanation of potential loss of range or precision depending on fix point conversion (incl. explanation of necessity to have integers)
* <span style="color:red;">Lines in red mean that I'm not a 100% sure for now + questions that I have</span> 

## Temporary Notes: 
Important Concepts: 
--> Source: [The three part blog series about SPDZ](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html) and [Morten Dahl's Blogs on SPDZ](https://mortendahl.github.io/2017/09/03/the-spdz-protocol-part1/)
* Functions are often refered to as circuits -> every function can be approx. represented by nested boolean circuits (combination of boolean gates) ??
* There are two types of SMPC: secret-sharing-based SMPC and circuit-gabling-based SMPC 
    * Basic protocol steps for secret-sharing-based SMPC see BLOG: https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html
    * Secret-sharing is mainly done via so called "Additive Secret Sharing" protocol, **Shamir's protocol** is a more complicated secret-sharing technique that doesn't only depend on additive splitting (reconstructable with a simple sum)
* SPDZ is a prominent example of secret-sharing-based SMPC 
  * It is considered to be the optimal method to use FHE in MPC according to authors of the original paper - as described in the blog(for the preprocessing)
  * The online phase is very good already (**TODO:** enumerate number of communications based on blog)
  * That's why mainly the offline phase - i.a. the preprocessing, generation of beaver triples and the big random numbers to serve to secret share values (the party having the real value also participates, other protocol than so far in tutorials where two dedicated units were used) - is being optimized. 
  * In the original SPDZ the preprocessing is done using Somewhat Homomorphic Encryption (SHE), for now no method exists to use instead FHE (**TODO:** How exactly? Are triples different for different functions?) in a competitive manner compared to other MPC protocols. 
  
--- 
 
--> Performance Metrics of Crypto-Protocols: Source [Morten Dahl's Blog on Private Image Analysis with MPC](https://mortendahl.github.io/2017/09/19/private-image-analysis-with-mpc/) 
* Communication Complexity: How many bits are being exchanged between participating parties? 
* Round Complexity: How many points of synchronization are needed in the protocol? (The more the likelier that one party is IDLE because it needs to wait for other party to finish some job e.g.)

---